In [1]:
import os
import json
import math
import price
import ratios
import requests
import momentum
import pyticker
import dividend
import questrade
import auto_email
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import pandas_datareader.data as web

## Global Variables

In [2]:
WATCHLIST = ['O']
MOMENTUM_PERIODS = [3,6,12,24,36,48,60]
DIV_GROWTH_RATE_THRESHOLD = 0.10
MOMENTUM_THRESHOLD = 0.50
MARKET_CAP_THRESHOLD = 100

## Create S&P 500 Data If Not Exists

In [3]:
if not os.path.exists('./dividend_stocks_sp500.json'):
    sp500 = pyticker.get_symbols_by_index('S&P 500')

    data = {'Symbol': [], 'Market_Cap (B)': []}


    count = 0

    for symbol in sp500:
        count += 1
        print(f"{symbol}: {count}/{len(sp500)}")
        try:
            market_cap = ratios.calculate_market_cap(symbol)
        except:
            continue

        if market_cap >= MARKET_CAP_THRESHOLD and dividend.exists_dividends(symbol):
            data['Symbol'].append(symbol)
            data['Market_Cap (B)'].append(market_cap)
            print(f"{symbol} has been added!")

    with open('./dividend_stocks_sp500.json', 'w') as fp:
        json.dump(data,fp)

LIN: 1/498
LIN has been added!
CCL: 2/498
SSE: 3/498
FTI: 4/498
MMM: 5/498
MMM has been added!
AXP: 6/498
AXP has been added!
AAPL: 7/498
AAPL has been added!
BA: 8/498
BA has been added!
CAT: 9/498
CAT has been added!
CVX: 10/498
CVX has been added!
CSCO: 11/498
CSCO has been added!
KO: 12/498
KO has been added!
DOW: 13/498
XOM: 14/498
XOM has been added!
GS: 15/498
GS has been added!
HD: 16/498
HD has been added!
IBM: 17/498
IBM has been added!
INTC: 18/498
INTC has been added!
JNJ: 19/498
JNJ has been added!
JPM: 20/498
JPM has been added!
MCD: 21/498
MCD has been added!
MRK: 22/498
MRK has been added!
MSFT: 23/498
MSFT has been added!
NKE: 24/498
NKE has been added!
PFE: 25/498
PFE has been added!
PG: 26/498
PG has been added!
TRV: 27/498
UNH: 28/498
UNH has been added!
VZ: 29/498
VZ has been added!
V: 30/498
V has been added!
WMT: 31/498
WMT has been added!
WBA: 32/498
DIS: 33/498
DIS has been added!
ATVI: 34/498
ADBE: 35/498
ADBE has been added!
AKAM: 36/498
ALXN: 37/498
GOOGL: 3

## Load Data

In [4]:
with open('./dividend_stocks_sp500.json', 'r') as fp:
    data = json.load(fp)
df = pd.DataFrame(data)
df.set_index('Symbol', inplace=True)
df

,Market_Cap (B)
Symbol,
LIN,150.697136
MMM,113.814921
AXP,122.632221
AAPL,2246.918769
BA,137.674863
...,...
SCHW,131.190678
TMO,191.674440
LMT,103.713852


## Update Watchlist

In [5]:
watchlist_data = {'Symbol': [], 'Market_Cap (B)': []}
for symbol in WATCHLIST:
    if len(WATCHLIST) == 1:
        watchlist_data['Symbol'] = symbol
        watchlist_data['Market_Cap (B)'] = ratios.calculate_market_cap(symbol)
    else:
        watchlist_data['Symbol'].append(symbol)
        watchlist_data['Market_Cap (B)'].append(ratios.calculate_market_cap(symbol))

df.reset_index(inplace=True)
df = df.append(watchlist_data, ignore_index=True)
df.set_index('Symbol', inplace=True)
df

,Market_Cap (B)
Symbol,
LIN,150.697136
MMM,113.814921
AXP,122.632221
AAPL,2246.918769
BA,137.674863
...,...
TMO,191.674440
LMT,103.713852
PEP,196.357440


## Dividend Growth and Momentum

In [6]:
for symbol in list(df.index):
    try:
        div_growth = dividend.calcualte_avg_dividend_growth(symbol,10)
        if div_growth == float('inf'):
            df.loc[symbol, 'Dividend_Growth'] = np.nan
        else:
            df.loc[symbol, 'Dividend_Growth'] = div_growth
    except:
        df.loc[symbol, 'Dividend_Growth'] = np.nan
        
    try:
        mom = momentum.calculate_equal_weight_momentum(symbol, MOMENTUM_PERIODS)
        df.loc[symbol,'Momentum'] = mom
    except:
        df.loc[symbol,'Momentum'] = np.nan

df.dropna(inplace=True)

In [7]:
df = df[(df['Dividend_Growth'] > DIV_GROWTH_RATE_THRESHOLD) & (df['Momentum'] > MOMENTUM_THRESHOLD)]
df

,Market_Cap (B),Dividend_Growth,Momentum
Symbol,,,
AAPL,2246.918769,0.238712,1.565962
CAT,127.023067,0.111480,1.099832
GS,118.378192,0.139685,0.714878
HD,345.566113,0.206233,0.759093
JPM,460.028387,0.468526,0.798693
MSFT,1918.438011,0.144052,1.413311
NKE,206.217306,0.137093,0.724494
UNH,373.336258,0.284789,0.813156
V,500.443105,0.253689,0.691086


## Export DataFrame

In [8]:
df.to_csv(r'./qualified_df.csv')

## Drawdowns from 52W High

In [9]:
for symbol in list(df.index):
    high = price.calculate_prev_max_high(symbol,252)
    curr_price = price.get_current_price(symbol)
    df.loc[symbol,'12M_High'] = high
    df.loc[symbol,'Current_Price'] = curr_price
    df.loc[symbol,'10%_Drop'] = high * 0.90
    df.loc[symbol,'20%_Drop'] = high * 0.80
    df.loc[symbol,'30%_Drop'] = high * 0.70
    df.loc[symbol,'50%_Drop'] = high * 0.5    
df

C:\Users\eshin\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\eshin\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Market_Cap (B),Dividend_Growth,Momentum,12M_High,Current_Price,15%_Drop,30%_Drop,50%_Drop
Symbol,,,,,,,,
AAPL,2246.918769,0.238712,1.565962,145.089996,134.389999,123.326497,101.562997,72.544998
CAT,127.023067,0.111480,1.099832,237.779999,230.339996,202.112999,166.445999,118.889999
GS,118.378192,0.139685,0.714878,356.850006,346.630005,303.322505,249.795004,178.425003
HD,345.566113,0.206233,0.759093,328.829987,320.589996,279.505489,230.180991,164.414993
JPM,460.028387,0.468526,0.798693,161.690002,151.250000,137.436502,113.183002,80.845001
MSFT,1918.438011,0.144052,1.413311,262.440002,261.970001,223.074002,183.708002,131.220001
NKE,206.217306,0.137093,0.724494,147.949997,132.110001,125.757497,103.564998,73.974998
UNH,373.336258,0.284789,0.813156,401.989990,394.500000,341.691492,281.392993,200.994995
V,500.443105,0.253689,0.691086,232.949997,229.910004,198.007497,163.064998,116.474998


In [10]:
df.head(10)

,Market_Cap (B),Dividend_Growth,Momentum,12M_High,Current_Price,15%_Drop,30%_Drop,50%_Drop
Symbol,,,,,,,,
AAPL,2246.918769,0.238712,1.565962,145.089996,134.389999,123.326497,101.562997,72.544998
CAT,127.023067,0.111480,1.099832,237.779999,230.339996,202.112999,166.445999,118.889999
GS,118.378192,0.139685,0.714878,356.850006,346.630005,303.322505,249.795004,178.425003
HD,345.566113,0.206233,0.759093,328.829987,320.589996,279.505489,230.180991,164.414993
JPM,460.028387,0.468526,0.798693,161.690002,151.250000,137.436502,113.183002,80.845001
MSFT,1918.438011,0.144052,1.413311,262.440002,261.970001,223.074002,183.708002,131.220001
NKE,206.217306,0.137093,0.724494,147.949997,132.110001,125.757497,103.564998,73.974998
UNH,373.336258,0.284789,0.813156,401.989990,394.500000,341.691492,281.392993,200.994995
V,500.443105,0.253689,0.691086,232.949997,229.910004,198.007497,163.064998,116.474998
